The code in this notebook was run on the Databricks notebook, following on from the code found in the dataframe cleaning notebook

In [ ]:
# import for performing window functions
from pyspark.sql.window import Window

In [ ]:
# find the most popular category in each country
df_pin.join(df_geo, df_pin.ind == df_geo.ind) \
    .groupBy("country", "category") \
    .agg(count("category").alias("category_count")) \
    .orderBy("country", "category_count", ascending=[True, False]) \
    .limit(50) \
    .display()

country,category,category_count
Afghanistan,education,9
Afghanistan,quotes,5
Afghanistan,mens-fashion,3
Afghanistan,tattoos,2
Afghanistan,art,2
Afghanistan,finance,2
Afghanistan,diy-and-crafts,2
Afghanistan,event-planning,2
Afghanistan,home-decor,1
Afghanistan,vehicles,1


In [ ]:
# find which was the most popular category each year between 2018 and 2022
df_pin.join(df_geo, df_pin.ind == df_geo.ind) \
    .withColumn("post_year", year("timestamp")) \
    .filter(col("post_year") >= 2018) \
    .filter(col("post_year") <= 2022) \
    .groupBy("post_year", "category") \
    .agg(count("category").alias("category_count")) \
    .orderBy("post_year", "category_count", ascending=[True, False]) \
    .display()

post_year,category,category_count
2018,diy-and-crafts,25
2018,christmas,23
2018,travel,23
2018,quotes,21
2018,education,20
2018,art,18
2018,tattoos,17
2018,beauty,15
2018,finance,14
2018,vehicles,13


In [ ]:
# find the user with the most followers in each country

# create partition by country and order by follower_count descending
windowCountry = Window.partitionBy("country").orderBy(col("follower_count").desc())
# join df_pin and df_geo on index
max_followers_by_country = \
    df_pin.join(df_geo, df_pin.ind == df_geo.ind) \
    .withColumn("row",row_number().over(windowCountry)) \
    .filter(col("row") == 1) \
    .select("country", "poster_name", "follower_count")

# get highest number of followers from all countries
max_followers_all_countries = max_followers_by_country.select(max("follower_count")).collect()[0][0]
# find the country with the user with most followers
country_with_max_followers = \
    max_followers_by_country \
    .select("*") \
    .where(col("follower_count") == max_followers_all_countries)

max_followers_by_country.limit(50).display()
country_with_max_followers.display()

country,poster_name,follower_count
Afghanistan,9GAG,3000000
Albania,The Minds Journal,5000000
Algeria,YourTango,942000
American Samoa,Mamas Uncut,8000000
Andorra,Teachers Pay Teachers,1000000
Angola,Tastemade,8000000
Anguilla,"Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog",92000
Antarctica (the territory South of 60 deg S),Refinery29,1000000
Antigua and Barbuda,Country Living Magazine,1000000
Argentina,Cheezburger,2000000


country,poster_name,follower_count
American Samoa,Mamas Uncut,8000000
Angola,Tastemade,8000000


In [ ]:
# find the most popular category for different age groups
# join df_pin and df_user, select relevant columns and create temp view for SQL query
df_pin.join(df_user, df_pin.ind == df_user.ind) \
    .select("category", "age") \
    .createOrReplaceTempView("category_age")
# SQL query, drop age column, then group by age_group and category
spark.sql("SELECT CASE \
            WHEN age between 18 and 24 then '18-24' \
            WHEN age between 25 and 35 then '25-35' \
            WHEN age between 36 and 50 then '36-50' \
            WHEN age > 50 then '50+' \
            END as age_group, * FROM category_age") \
    .drop("age") \
    .groupBy("age_group", "category") \
    .agg(count("category").alias("category_count")) \
    .orderBy("age_group", "category_count", ascending=[True, False]) \
    .display()

age_group,category,category_count
18-24,art,68
18-24,tattoos,62
18-24,christmas,48
18-24,diy-and-crafts,47
18-24,quotes,47
18-24,education,44
18-24,mens-fashion,31
18-24,finance,28
18-24,travel,27
18-24,event-planning,26
